<a href="https://colab.research.google.com/github/fawazshah/Reddit-Analysis/blob/main/3_sentiment_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import pandas as pd

nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Loading data

In [2]:
submissions_lib_dem_con_rep_url = 'https://raw.githubusercontent.com/fawazshah/Reddit-Analysis/master/data/assembled-data/submissions_top300_year_liberal_democrats_conservative_republicans.tsv'
submissions_lib_dem_con_rep_df = pd.read_csv(submissions_lib_dem_con_rep_url, sep='\t')

comments_lib_dem_con_rep_url = 'https://raw.githubusercontent.com/fawazshah/Reddit-Analysis/master/data/assembled-data/comments_top300_year_liberal_democrats_conservative_republicans.tsv'
comments_lib_dem_con_rep_df = pd.read_csv(comments_lib_dem_con_rep_url, sep='\t')

In [3]:
submissions_lib_dem_con_rep_df

,submission id,subreddit,article headline,article body,bias
0,l6a0q7,liberal,"Republicans now 'shocked, shocked' that there'...","© Greg Nash Republicans now 'shocked, shocked'...",left
1,jxxs8b,liberal,Georgia certifies election results confirming ...,Georgia Secretary of State Ben Raffensperger h...,left
2,kuscob,liberal,Report: QAnon Congresswoman Was Live-Tweeting ...,"Domestic Terrorist: Rep. Lauren Boebert, a new...",left
3,j2lufw,liberal,"More than 175 current, former law enforcement ...",EXCLUSIVE: More than 175 current and former la...,left
4,l8m3a8,liberal,GOP group launches billboards demanding Cruz a...,GOP campaigners have called on senators Ted Cr...,left
...,...,...,...,...,...
1129,kd595w,republicans,41 days after election day in the United State...,a Google company\n\nDeliver and maintain servi...,right
1130,hz72fr,republicans,98 more days until liberal tears flow...AGAIN!...,NaN,right
1131,hpjde4,republicans,Don Lemon accidentally tells the truth about B...,a Google company\n\nDeliver and maintain servi...,right
1132,hgah1j,republicans,Pelosi: “I Do Not” Apologize For Accusing Repu...,a Google company\n\nDeliver and maintain servi...,right


In [4]:
comments_lib_dem_con_rep_df

,comment id,submission id,subreddit,comment body,bias
0,gkzccbm,l6a0q7,liberal,"Hey Republican geniuses, I'll bet you were als...",left
1,gkzg91o,l6a0q7,liberal,The deficit exploded after the republican tax ...,left
2,gkzfown,l6a0q7,liberal,The Republican Party is a fucking cancer on ou...,left
3,gkz73xz,l6a0q7,liberal,"I wish I had gold to give you, just for the ti...",left
4,gkzhm11,l6a0q7,liberal,"it's not these politicians that really bug me,...",left
...,...,...,...,...,...
46168,gj8n6cn,kx5uz2,republicans,I don't understand this way of thinking. Sure ...,right
46169,gi4pnio,kqfpzu,republicans,"Come on guys, when Democrats were pretending t...",right
46170,gi4a4w8,kqfpzu,republicans,"If you believe the results are a sham, and you...",right
46171,gfv271m,kd595w,republicans,"My dudes... It's over. Keep hearing ""Just wa...",right


### Data preprocessing

In [5]:
print(submissions_lib_dem_con_rep_df['article body'].isna().sum())

609


In [6]:
# Remove NA article body values

submissions_lib_dem_con_rep_df.dropna(subset=['article body'], inplace=True)
submissions_lib_dem_con_rep_df.reset_index(drop=True, inplace=True)

In [7]:
submissions_lib_dem_con_rep_df

,submission id,subreddit,article headline,article body,bias
0,l6a0q7,liberal,"Republicans now 'shocked, shocked' that there'...","© Greg Nash Republicans now 'shocked, shocked'...",left
1,jxxs8b,liberal,Georgia certifies election results confirming ...,Georgia Secretary of State Ben Raffensperger h...,left
2,kuscob,liberal,Report: QAnon Congresswoman Was Live-Tweeting ...,"Domestic Terrorist: Rep. Lauren Boebert, a new...",left
3,j2lufw,liberal,"More than 175 current, former law enforcement ...",EXCLUSIVE: More than 175 current and former la...,left
4,l8m3a8,liberal,GOP group launches billboards demanding Cruz a...,GOP campaigners have called on senators Ted Cr...,left
...,...,...,...,...,...
520,kuk5xm,republicans,Arnold on Twitter: My message to my fellow Ame...,JavaScript is not available.\n\nWe’ve detected...,right
521,kqfpzu,republicans,Worse Than Treason: No amount of rationalizing...,"Today, the “sedition caucus” includes at least...",right
522,kd595w,republicans,41 days after election day in the United State...,a Google company\n\nDeliver and maintain servi...,right
523,hpjde4,republicans,Don Lemon accidentally tells the truth about B...,a Google company\n\nDeliver and maintain servi...,right


### Simple text preprocessing

In [8]:
def preprocess(sentence):

    # No lowercasing since upper-case words will indicate sentiment (anger or joy)
    # Also no punctuation removal since ! and ? can indicate sentiment

    # Whitespace removal
    whitespace = '''\n\t'''

    for ch in sentence: 
        if ch in whitespace:
            sentence = sentence.replace(ch, " ")
    
    return sentence

In [9]:
submissions_lib_dem_con_rep_df['article headline'] = submissions_lib_dem_con_rep_df['article headline'].apply(preprocess)
submissions_lib_dem_con_rep_df['article body'] = submissions_lib_dem_con_rep_df['article body'].apply(preprocess)
comments_lib_dem_con_rep_df['comment body'] = comments_lib_dem_con_rep_df['comment body'].apply(preprocess)

In [10]:
submissions_lib_dem_con_rep_df

,submission id,subreddit,article headline,article body,bias
0,l6a0q7,liberal,"Republicans now 'shocked, shocked' that there'...","© Greg Nash Republicans now 'shocked, shocked'...",left
1,jxxs8b,liberal,Georgia certifies election results confirming ...,Georgia Secretary of State Ben Raffensperger h...,left
2,kuscob,liberal,Report: QAnon Congresswoman Was Live-Tweeting ...,"Domestic Terrorist: Rep. Lauren Boebert, a new...",left
3,j2lufw,liberal,"More than 175 current, former law enforcement ...",EXCLUSIVE: More than 175 current and former la...,left
4,l8m3a8,liberal,GOP group launches billboards demanding Cruz a...,GOP campaigners have called on senators Ted Cr...,left
...,...,...,...,...,...
520,kuk5xm,republicans,Arnold on Twitter: My message to my fellow Ame...,JavaScript is not available. We’ve detected t...,right
521,kqfpzu,republicans,Worse Than Treason: No amount of rationalizing...,"Today, the “sedition caucus” includes at least...",right
522,kd595w,republicans,41 days after election day in the United State...,a Google company Deliver and maintain service...,right
523,hpjde4,republicans,Don Lemon accidentally tells the truth about B...,a Google company Deliver and maintain service...,right


In [11]:
comments_lib_dem_con_rep_df

,comment id,submission id,subreddit,comment body,bias
0,gkzccbm,l6a0q7,liberal,"Hey Republican geniuses, I'll bet you were als...",left
1,gkzg91o,l6a0q7,liberal,The deficit exploded after the republican tax ...,left
2,gkzfown,l6a0q7,liberal,The Republican Party is a fucking cancer on ou...,left
3,gkz73xz,l6a0q7,liberal,"I wish I had gold to give you, just for the ti...",left
4,gkzhm11,l6a0q7,liberal,"it's not these politicians that really bug me,...",left
...,...,...,...,...,...
46168,gj8n6cn,kx5uz2,republicans,I don't understand this way of thinking. Sure ...,right
46169,gi4pnio,kqfpzu,republicans,"Come on guys, when Democrats were pretending t...",right
46170,gi4a4w8,kqfpzu,republicans,"If you believe the results are a sham, and you...",right
46171,gfv271m,kd595w,republicans,"My dudes... It's over. Keep hearing ""Just wa...",right


### Sentiment analysis

In [12]:
subreddits = ['liberal', 'democrats', 'conservative', 'republicans']

In [13]:
# We will store only the compound (overall) sentiment

results = {}

for subreddit in subreddits:
    results[subreddit] = {}
    results[subreddit]['article headlines'] = []
    results[subreddit]['article bodies'] = []
    results[subreddit]['comment bodies'] = []

In [14]:
sia = SentimentIntensityAnalyzer()

for i, row in submissions_lib_dem_con_rep_df.iterrows():
    subreddit = row['subreddit']
    headline = row['article headline']
    body = row['article body']
    results[subreddit]['article headlines'].append(sia.polarity_scores(headline)['compound'])
    results[subreddit]['article bodies'].append(sia.polarity_scores(body)['compound'])

for i, row in comments_lib_dem_con_rep_df.iterrows():
    subreddit = row['subreddit']
    comment = row['comment body']
    results[subreddit]['comment bodies'].append(sia.polarity_scores(comment)['compound'])

In [15]:
for subreddit in subreddits:
    print(subreddit)
    headline_sentiments = results[subreddit]['article headlines']
    article_body_sentiments = results[subreddit]['article bodies']
    comment_sentiments = results[subreddit]['comment bodies']
    print(f"Headline sentiment: {sum(headline_sentiments) / len(headline_sentiments)}")
    print(f"Article body sentiment: {sum(article_body_sentiments) / len(article_body_sentiments)}")
    print(f"Comment sentiment: {sum(comment_sentiments) / len(comment_sentiments)}")
    print()

liberal
Headline sentiment: -0.12769147286821703
Article body sentiment: -0.035938372093023294
Comment sentiment: -0.06145019904458616

democrats
Headline sentiment: -0.14734285714285714
Article body sentiment: 0.4114928571428571
Comment sentiment: 0.03598296022201663

conservative
Headline sentiment: -0.05971562500000001
Article body sentiment: -0.014589374999999991
Comment sentiment: 0.019022813248099398

republicans
Headline sentiment: -0.02522911392405063
Article body sentiment: 0.11647974683544304
Comment sentiment: -0.06247160493827159

